<a href="https://colab.research.google.com/github/M0Leo/job-fit-analyzer/blob/flask/integration_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request, jsonify
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd


In [ ]:
app = Flask("Content_Based_System")

In [ ]:
model_title = RandomForestClassifier(random_state=40)
model_location = RandomForestClassifier(random_state=42)
tfidf = TfidfVectorizer()

In [ ]:
file_path = 'jobss.csv'
jobs_data = pd.read_csv(file_path)
jobs_data.dropna(subset=['Job Title'], inplace=True)


In [ ]:
X = tfidf.fit_transform(jobs_data['Key Skills'].astype(str))


In [ ]:
y_title = jobs_data['Job Title']
model_title.fit(X, y_title)

In [ ]:
y_location = jobs_data['Location']
model_location.fit(X, y_location)

In [ ]:
@app.route('/predict', methods=['POST'])
def predict_job_title_and_location():
    data = request.get_json()
    skills = data['skills']
    top_n = data.get('top_n', 5)
    relevance_threshold = data.get('relevance_threshold', 0.08)

    skills_transformed = tfidf.transform([skills.lower()])

    probs_title = model_title.predict_proba(skills_transformed)[0]
    top_n_indices_title = np.argsort(probs_title)[-top_n:]
    top_n_probs_title = probs_title[top_n_indices_title]
    relevant_titles = [model_title.classes_[i] for i, prob in zip(top_n_indices_title, top_n_probs_title) if prob >= relevance_threshold]

    predicted_title = np.random.choice(relevant_titles) if relevant_titles else 'No relevant title found'

    probs_location = model_location.predict_proba(skills_transformed)[0]
    top_n_indices_location = np.argsort(probs_location)[-top_n:]
    predicted_location = np.random.choice([model_location.classes_[i] for i in top_n_indices_location])

    return jsonify({
        'predicted_title': predicted_title,
        'predicted_location': predicted_location
    })

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app 'Content_Based_System'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
